In [1]:
import os
#https://github.com/langchain-ai/langgraph/blob/main/examples/rag/langgraph_agentic_rag.ipynb
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
import pyowm
from langchain_community.utilities import OpenWeatherMapAPIWrapper
from langgraph.graph import Graph
from langgraph.graph.message import add_messages 
from langchain_core.messages import AIMessage, HumanMessage, BaseMessage
from typing import TypedDict, Annotated, Sequence
from langgraph.graph import StateGraph, END
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain_community.tools.openweathermap import OpenWeatherMapQueryRun
from langchain_core.utils.function_calling import convert_to_openai_function
from langgraph.prebuilt import ToolInvocation
import json
from langchain_core.messages import FunctionMessage
from langgraph.prebuilt import ToolExecutor
# Now you can access your environment variables using os.environ

load_dotenv(dotenv_path="../.env")

True

In [2]:
# Set the model as ChatOpenAI
model = ChatOpenAI(temperature=0.5) 

weather = OpenWeatherMapAPIWrapper()

tools = [OpenWeatherMapQueryRun()]

functions = [convert_to_openai_function(function) for function in tools]

model = model.bind_functions(functions)

In [5]:

class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]

tool_executor = ToolExecutor(tools)


/tmp/ipykernel_1204827/2571114034.py:4: LangGraphDeprecationWarning: ToolExecutor is deprecated as of version 0.2.0 and will be removed in 0.3.0. Use langgraph.prebuilt.ToolNode instead.
  tool_executor = ToolExecutor(tools)


In [15]:
def function_1(state):
    messages = state["messages"]
    #user_input = messages[-1].content
    response = model.invoke(messages)
    print(f"Node1 response: {response}")
    return {"messages": [response]}    

In [24]:
def function_2(state):
    messages = state['messages']
    last_message = messages[-1] # this has the query we need to send to the tool provided by the agent

    parsed_tool_input = json.loads(last_message.additional_kwargs["function_call"]["arguments"])
    print(f"Parsed tool input: {parsed_tool_input}")

    # We construct an ToolInvocation from the function_call and pass in the tool name and the expected str input for OpenWeatherMap tool
    action = ToolInvocation(
        tool=last_message.additional_kwargs["function_call"]["name"],
        tool_input=parsed_tool_input['location'],
    )
    
    # We call the tool_executor and get back a response
    response = tool_executor.invoke(action)

    # We use the response to create a FunctionMessage
    function_message = FunctionMessage(content=str(response), name=action.tool)

    # We return a list, because this will get added to the existing list
    return {"messages": [function_message]}

In [25]:
def where_to_go(state):
    messages = state['messages']
    last_message = messages[-1]
    
    if "function_call" in last_message.additional_kwargs:
        return "continue"
    else:
        return "end"

In [26]:
# Or you could import StateGraph and pass AgentState to it
from langgraph.graph import StateGraph, END
workflow = StateGraph(AgentState)

workflow.add_node("agent", function_1)
workflow.add_node("tool", function_2)

# The conditional edge requires the following info below.
# First, we define the start node. We use `agent`.
# This means these are the edges taken after the `agent` node is called.
# Next, we pass in the function that will determine which node is called next, in our case where_to_go().

workflow.add_conditional_edges("agent", where_to_go,{   # Based on the return from where_to_go
                                                        # If return is "continue" then we call the tool node.
                                                        "continue": "tool",
                                                        # Otherwise we finish. END is a special node marking that the graph should finish.
                                                        "end": END
                                                    }
)

# We now add a normal edge from `tools` to `agent`.
# This means that if `tool` is called, then it has to call the 'agent' next. 
workflow.add_edge('tool', 'agent')

# Basically, agent node has the option to call a tool node based on a condition, 
# whereas tool node must call the agent in all cases based on this setup.

workflow.set_entry_point("agent")


app = workflow.compile()

In [28]:
inputs = {"messages": [HumanMessage(content="what is the temperature in las vegas")]}
#app.invoke(inputs)
inputs = {"messages": [HumanMessage(content="what is the temperature in las vegas")]}
for output in app.stream(inputs):
    # stream() yields dictionaries with output keyed by node name
    for key, value in output.items():
        print(f"Output from node '{key}':")
        print("---")
        print(value)
    print("\n---\n")

Node1 response: content='' additional_kwargs={'function_call': {'arguments': '{"location":"Las Vegas"}', 'name': 'open_weather_map'}, 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 79, 'total_tokens': 95}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None} id='run-b821a752-25c1-40a3-9c21-05f3744e0b1a-0' usage_metadata={'input_tokens': 79, 'output_tokens': 16, 'total_tokens': 95}
Output from node 'agent':
---
{'messages': [AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"location":"Las Vegas"}', 'name': 'open_weather_map'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 79, 'total_tokens': 95}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-b821a752-25c1-40a3-9c21-05f3744e0b1a-0', usage_metadata={'input_tokens': 79, 'output_tok

/tmp/ipykernel_1204827/766937530.py:9: LangGraphDeprecationWarning: ToolInvocation is deprecated as of version 0.2.0 and will be removed in 0.3.0. Use langgraph.prebuilt.ToolNode instead.
  action = ToolInvocation(


Node1 response: content='The current temperature in Las Vegas is 28.47°C.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 202, 'total_tokens': 216}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-60d1a06f-16c2-43a6-94ae-54e2d9422c1f-0' usage_metadata={'input_tokens': 202, 'output_tokens': 14, 'total_tokens': 216}
Output from node 'agent':
---
{'messages': [AIMessage(content='The current temperature in Las Vegas is 28.47°C.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 202, 'total_tokens': 216}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-60d1a06f-16c2-43a6-94ae-54e2d9422c1f-0', usage_metadata={'input_tokens': 202, 'output_tokens': 14, 'total_tokens': 216})]}

---

